In [1]:
# Standard Libraries of Python
import sys
import time
from collections import Counter

# Dependencies
import pandas as pd
import numpy as np

# Libraries proper of this Proyect
from database.clean_database import database
from data_analisys.legacy_data_functions import draw_generator, numbers_boolean, first_df_bool
from data_analisys.numbers_analisys import analisys

ModuleNotFoundError: No module named 'legacy_data_functions'

In [ ]:
"""Constants of the main code for numbers analisys"""
# Main Database to calculate the first DataFrame main_df_counts
db = pd.read_parquet('database/db.parquet')
db_slice = db
lenght = len(db)

# Array of numbers
total_numbers = np.arange(1,51)

# Create a template DataFrame with all values set to False
skip_winners_bool = pd.DataFrame(False, columns=[str(i) for i in range(1, 51)], index=range(len(db)))
    
# Fill in the True values
main_boolean_df = numbers_boolean(db, skip_winners_bool, total_numbers)

# Main DataFrame to be updated with new draws in function analisys
main_df_counts = first_df_bool(db, total_numbers)

In [ ]:
start_time = time.time()

succes = []
failure = []
for draw in draw_generator(lenght):
    db_resultados = db_slice.head(draw)
    recommended_numbers, not_recommended_numbers, main_counts = analisys(db_resultados, main_boolean_df, main_df_counts)
    main_df_counts = main_counts
    row = db.loc[draw,['nro1','nro2','nro3','nro4','nro5']]
    column = recommended_numbers.loc[:, 'Numbers']
    result = column.isin(row).sum()
    succes.append(result)
    failure.append(5-result)
    sys.stdout.write(f"\ri = {draw}")
    sys.stdout.flush()
    
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")

In [ ]:
start_time = time.time()

random_succes = []
random_failure = []
for draw in draw_generator(lenght):
    db_resultados = db_slice.head(draw)
    row = db.loc[draw,['nro1','nro2','nro3','nro4','nro5']]
    random_numbers = np.random.choice(range(1,51), size=14, replace=False)
    result = np.isin(row, random_numbers).sum()
    random_succes.append(result)
    random_failure.append(5-result)
    sys.stdout.write(f"\ri = {draw}")
    sys.stdout.flush()

end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")

In [ ]:
c = Counter(succes)
x = Counter(failure)
y = Counter(random_succes)
z = Counter(random_failure)

# Imprimir la cantidad de aciertos para cada número de aciertos posibles
for i in range(6):
    hits = (succes.count(i)/len(succes))*100
    print(f"{i} hits: {c[i]}\n{round(hits,2)}%")

for e in range(6):
    nohits = (random_succes.count(e)/len(random_succes))*100
    print(f"{e} hits: {y[e]}\n{round(nohits,2)}%")